# Recurrent Neural Networks

We'll now go beyond the vanilla dense neural network we designed last time and look at some more advanced and specialized neural network architectures. Our first stop is the world of sequenctial data and Recurrent Neural Networks (RNNs). In this guide, we'll explore RNNs and their applications. Then, next time, we'll tackle some of the limitations of RNNs and how to address them.

RNNs are a class of neural networks that are tailored for sequential data such as text, audio and time series data. To appreciate the value of RNNs for such data, it's important to first understand the limitations of the standard dense neural network.

## Limitations of Dense Neural Networks

While dense neural networks are powerful and versatile, they come with certain shortcomings, especially when handling sequential data:

1. **No Sense of Order**: Dense neural networks treat inputs as independent entities. For instance, if we feed a sentence to a dense neural network, it will process each word in isolation, without any sense of order or context. This limitation is especially relevant for sequential data, where the order of elements is often crucial for their interpretation.
    - For example, a dense neural network might treat the sentences "Man eats shark" and "Shark eats man" as equivalent, even though they have very different meanings.
2. **Fixed Input and Output Sizes**: Dense networks require a predetermined size for their input and output layers. This can be limiting for tasks where the length of the input sequence might vary, such as processing sentences of different lengths in natural language tasks.
3. **Parameter Efficiency**: For large input sizes, the number of parameters in a dense neural network can grow exponentially, leading to overfitting and increased computational demands.

## In Comes Recurrent Neural Networks

RNNs where introduced to to address these challenges. Unlike dense neural networks that require fixed input and output sizes, RNNs are inherently designed to process sequences of variable lengths. This flexibility arises from their unique architecture that loops over the sequence, handling just one element at a time (RNNs should really be tought of as a looping neural network). This looping design means that whether a sentence has five words or fifty, RNNs can process it without the need for reshaping or truncating the input.

Moreover, as the RNN process sequences step-by-step, it maintains a hidden state that carries information from prior steps. This "memory" ensures that context is preserved. For instance, in a sentence, the meaning of a word can often be influenced by preceding words. RNNs, by virtue of this internal memory, can capture such dependencies, ensuring that every input is understood not just in isolation, but in the context of its preceding elements..

First, we'll import the necessary libraries. Next, we'll explore the the `ValueArray` class and its purpose. After that, we'll dive into RNN architectures and conclude by building and training a RNN from the ground up.

In [1]:
# To allow importing modules from the dlafs directory, can be ignored
import sys
import os

# Get the absolute path of the 'src' directory
src_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add 'src' directory to the Python path
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
import random

import numpy as np

import dlafs
from dlafs import Value, ValueArray
from dlafs.nn import BaseNeuron, Module, Layer

np.random.seed(42)
random.seed(42)

%matplotlib inline

## ValueArray

Moving on forward, we'll be working with neural network architectures that requires multi-dimensional arrays (i.e. multidimensional lists of numbers). Since this is a pure python neural networks from scratch guide, we'll have to use our own implementation of multi-dimensional arrays instead of other libraries such as `numpy`.

To that end, we'll use the `ValueArray` class to represent multi-dimensional arrays. The `ValueArray` class is implemented using
standard python lists and can only hold `Value` objects. The API is quite simple since it doesn't implement any mathematical operations. Instead, it's designed to be used as a container for data, and only provides methods for getting/setting values and converting to/from lists and numpy arrays.

We'll use the rest of this section to explore the API of the `ValueArray` class.

In [3]:
# Creating a ValueArray of zeros
arr = ValueArray.zeros(shape=(3, 2), label='test')
print(arr)
print(arr.shape)

ValueArray(
    [[0, 0],
     [0, 0],
     [0, 0]],
    label='test'
)
(3, 2)


In [4]:
# Getting values
print(arr[0, 0])  # Returns a Value object
print(arr[0,])
print(arr[:, 0])

Value(0, label='test_0_0')
ValueArray(
    [0, 0]
)
ValueArray(
    [0, 0, 0]
)


In [5]:
# Setting values
arr[0, 0] = 1  # Converts to a Value object
arr[2, :] = [3, 3]
arr

ValueArray(
    [[1, 0],
     [0, 0],
     [3, 3]],
    label='test'
)

In [6]:
# To and from numpy arrays
np_arr = np.arange(24).reshape(3, 2, 2, 2)
v_arr = ValueArray.from_numpy(np_arr)
print(v_arr[0, 0, :, :])
print(v_arr[0, 0, :, :].to_numpy())

ValueArray(
    [[0, 1],
     [2, 3]]
)
[[0 1]
 [2 3]]


In [7]:
# To and from lists
py_list = [[1.41, 2.22], [3.3, 4.4], [5.5, 6.6]]
v_arr = ValueArray(py_list)

print(v_arr)

print(v_arr.to_list())  # Returns a nested list of floats or ints
print(v_arr.values)  # Returns a nested list of Value objects

ValueArray(
    [[1.41, 2.22],
     [ 3.3,  4.4],
     [ 5.5,  6.6]]
)
[[1.41, 2.22], [3.3, 4.4], [5.5, 6.6]]
[[Value(1.41), Value(2.22)], [Value(3.3), Value(4.4)], [Value(5.5), Value(6.6)]]


In [8]:
# Creating a ValueArray of random values
# sampled from a normal dist with mean 0 and std 1.
print(ValueArray.random_normal(shape=(3, 2)))

# Creating a ValueArray of random values
# sampled from a uniform dist between 0 and 5.
ValueArray.random_uniform(shape=(3, 2), low=0, high=5)

ValueArray(
    [[ -0.14409, -0.172904],
     [-0.111316,  0.701984],
     [-0.127588, -1.497353]]
)


ValueArray(
    [[4.460898, 0.434694],
     [2.109609, 0.148986],
     [ 1.09319, 2.526776]]
)

## The RNN Neuron

The RNN neuron is the basic building block of RNNs. It's similar to the standard neuron we've seen before, but with a few key differences. Let's start by looking at the standard neuron again:

<img src="img/neuron_model.png" alt="Standard Neuron" width="600"/>

The standard neuron takes a list of inputs $x_i$, multiplies it with the corresponding weights $w_i$, sums them together, and adds a bias $b$ to the result. Lastly, it passes the weighted sum through an activation function $f$ to produce the output $a$.

$$
a = f(\sum_{i=1}^{n} w_i x_i + b)
$$

A RNN neuron is illustrated below (Note that superscripts denote the time step or what inputs the weights are multiplied with):

<img src="img/recurrent_neuron_model.png" alt="RNN Neuron" width="600"/>

As you can see, the RNN neuron is similar, but it has a key difference: it has a hidden state $a^{\braket{t}}$ that is passed from one timestep (denoted $\braket{t}$) to the next. 

This hidden state is computed by combining the current input $x^{\braket{t}}$ and the previous hidden state $a^{\braket{t-1}}$. The inputs are multiplied by their corresponding weights $w^x$ and $w^a$ and summed together (+ a bias $b$), and then passed through an activation function to compute the hidden state $a^{\braket{t}}$. The hidden state is then passed as an input to the next timestep $t+1$.

This gives us the following equation for the hidden state:

$$
a^{\braket{t}}_i = f(\sum_{i=1}^{n^x} w^x_i x^{\braket{t}}_i + \sum_{j=1}^{n^a}w^a_j a^{\braket{t-1}}_j + b)
$$

Where the activation function $f$ is usually $tanh$.

The formula above outputs the hidden state for a single neuron $a^{\braket{t}}_i$. However, it takes $n^a$ inputs from the previous hidden state, meaning that we'll need $n^a$ neurons to compute the new hidden state. We'll call this collection of neurons the recurrent layer.

## The Basic RNN Architecture

A RNN uses the same recurrent layer in each timestep, passing the hidden state from one timestep to the next. This allows the RNN to loop through a (arbitrary-length) sequence of inputs one at a time. The hidden state acts as the neuron's memory, allowing it to retain information from previous inputs. The looping design is illustrated below:

<img src="img/rnn_simplified.png" alt="RNN Architecture" width="800"/>

One thing we haven't mentioned yet is how the output $\hat{y}^{\braket{t}}$ is computed. We can simply compute $\hat{y}^{\braket{t}}$ by passing the hidden state $a^{\braket{t}}$ as the input to a standard neuron.

Lastly, it should be noted that the first timestep $\braket{t}=0$ is a special case. At this timestep, there is no previous hidden state $a^{\braket{t-1}}$ to use. Instead, the hidden state $a^{\braket{-1}}$ is initialized to a list of zeros.

### Different Types of RNNs

While the image above gives a general idea of RNNs, it's essential to recognize that not all RNNs are identical. The output $\hat{y}$​ might not be computed at each timestep $\braket{t}$. This is because there are different RNNs designs, each suited for different tasks. 

<img src="img/rnn_types.jpeg" alt="RNN Types" width="900"/>

**One-to-One**: This is the standard neural network architecture, where one input is processed to produce one output. It doesn't utilize sequences and is more akin to traditional feed-forward neural networks.

**One-to-Many**: This type of RNN is designed to take one input and produce a sequence of outputs. It's commonly used in tasks like image captioning, where an image (single input) is used to generate a sentence (sequence of words).

**Many-to-One**: Here, a sequence of inputs produces a single output. This architecture is typically employed in tasks like sentiment analysis, where a sequence of words in a sentence is used to determine a single sentiment score or category.

**Many-to-Many(left)**: This design processes a sequence of inputs to produce a sequence of outputs where the lengths of input and output sequences are the same. It's often used in tasks like part-of-speech tagging, where each word in an input sentence is tagged with its corresponding part of speech.

**Many-to-Many (right)**: Also known as "sequence-to-sequence", this type of RNN takes a sequence of inputs and produces a potentially different-length sequence of outputs. This architecture is crucial for tasks like machine translation, where an input sentence in one language is translated to an output sentence in another language, and the lengths of the two sentences may differ.

We'll start with implementing the Recurrent Neuron. It takes the number of inputs and the size of the hidden state (number of recurrent neurons) as parameters. It also takes an activation function as a parameter, which defaults to `tanh` if none is provided.
(The last parameter `neuron_id` can be ignored since it's only used for labeling `Value` objects)

The init method initialized the weights to random numbers, like in the standard neuron. While the call method outputs the new hidden state as described above.

In [9]:
class RecurrentNeuron(BaseNeuron):

    def __init__(self, num_inputs, hidden_size, activation='tanh', neuron_id=None):
        if neuron_id is not None:
            neuron_id = f'_{neuron_id}'
        # Initialize the weights and bias
        self.wx = ValueArray.random_normal(shape=(num_inputs, ), label=f'wx{neuron_id}',
                                           mean=0, std=2/(num_inputs+hidden_size))
        self.wa = ValueArray.random_normal(shape=(hidden_size, ), label=f'wa{neuron_id}',
                                           mean=0, std=1/(hidden_size))
        self.ba = Value(0, label='ba')
        self._activation = activation

    def __call__(self, x, a):
        """The forward pass of a single neuron"""
        if len(x) != self.wx.shape[0]:
            raise ValueError(f'Expected {self.wx.shape[0]} inputs, got {len(x)}')
        if not len(a) == self.wa.shape[0]:
            raise ValueError(f'Expected {self.wa.shape[0]} hidden inputs, got {len(a)}')
        a = ValueArray(a, label='a')

        zx = sum(wx_i * x_i for wx_i, x_i in zip(self.wx, x))
        za = sum(wa_i * a_i for wa_i, a_i in zip(self.wa, a))
        z = zx + za + self.ba

        out = self.activation(z)
        return out

    def parameters(self):
        """Return the weights and bias as a list"""
        return self.wx.values + self.wa.values + [self.ba]

    def __repr__(self):
        return f'RNN-Neuron({self.wx.shape[0]})'


neuron = RecurrentNeuron(num_inputs=2, hidden_size=1, activation='tanh')
print(neuron)

RNN-Neuron(2)


We can use this single recurrent neuron to make predictions on a sequence of inputs. However, the hidden state can only be of size one (i.e. a single neuron).

In [10]:
inputs = [[Value(1, label='x_1'), Value(2, label='x_2')], [Value(1, label='x_1'), Value(2, label='x_2')]]
a_0 = [0,]
a_1 = neuron(inputs[0], a_0)
print(a_1)
a_2 = neuron(inputs[1], [a_1,])
print(a_2)

Value(0.526363)
Value(0.193986)


To increase the size of the hidden state, we'll need to use multiple recurrent neurons organized in a layer. We'll call this layer the `RecurrentLayer`. 

The init method takes the same arguments as the `RecurrentNeuron` (number of inputs, hidden state size, activation) and initializes a list of `RecurrentNeuron` objects.

The call method takes a sequence of inputs in the shape *(sequence length, number of inputs)* and outputs a sequence of hidden states in the shape *(sequence length, hidden state size)*. It computes the hidden states by initializing the first hidden state to a list of zeros, and then looping through the inputs, passing the current input and the previous hidden state to each neuron in the layer. It then appends the new hidden state to a list of hidden states and returns it.

In [11]:
class RecurrentLayer(Module):

    def __init__(self, num_inputs, hidden_size, activation='tanh'):
        self.hidden_size = hidden_size
        self.neurons = [RecurrentNeuron(num_inputs, hidden_size, activation, neuron_id=i) for i in range(hidden_size)]

    def __call__(self, x):
        x = ValueArray(x)
        # Check that the number of inputs equals the number of weights
        if not x.shape[1] == self.neurons[0].wx.shape[0]:
            raise ValueError(f'Expected {self.neurons[0].wx.shape[0]} inputs, got {x.shape[1]}')

        a_t = ValueArray.zeros(shape=(self.hidden_size,), label='a_t')  # Initialize hidden state to zeros
        a = []
        for x_t in x:
            a_t = [n(x_t, a_t) for n in self.neurons]
            a.append(a_t)
        return ValueArray(a)

    def parameters(self):
        """Return the weights and bias as a list"""
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"RecurrentLayer({self.neurons})"

layer = RecurrentLayer(num_inputs=3, hidden_size=4, activation='tanh')

We're now only missing one piece of the puzzle: the output neuron. We can create a RNN by combining the `RecurrentLayer` with layers of standard neurons. We'll call this combination the `RecurrentNN` class. Note that we can't use the `VanillaNN` class we created last time since it's not designed to handle sequences of inputs.

In [12]:
class RecurrentNN(Module):

    def __init__(self, layers):
        self.layers = layers

    def __call__(self, x):
        x = ValueArray(x)
        for layer in self.layers:
            if x.dim() > 1 and not isinstance(layer, RecurrentLayer):
                new_x = []
                for x_t in x:
                    new_x.append(layer(x_t))
                x = ValueArray(new_x)
            else:
                x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        layers_str = ',\n  '.join([str(layer) for layer in self.layers])
        return f"RecurrentNN([\n  {layers_str}\n])"

In [13]:
layers = [
    RecurrentLayer(num_inputs=2, hidden_size=5, activation='tanh'),
    Layer(5, 1, activation='sigmoid')
]
rnn = RecurrentNN(layers)
print(rnn)

RecurrentNN([
  RecurrentLayer([RNN-Neuron(2), RNN-Neuron(2), RNN-Neuron(2), RNN-Neuron(2), RNN-Neuron(2)]),
  Layer([SigmoidNeuron(5)])
])


There you have it! We've now implemented a basic RNN from scratch. We can use it to make predictions on sequences of inputs of arbitrary length. Let's put it to the test on a simple task. 

The task is to predict classify if the sum of the inputs is increasing at every timestep. Where the input for each timestep is a list of two numbers. For instance, the sequence `[[1, 2], [3, 4]]` should be classified as `True` (or 1) while `[[1, 2], [0, 2]]` should be classified as `False` (or 0).

In [14]:
inputs = [
    [(1, 3), (4, 2), [5, 7], [-2, 15]],
    [(4, 2), (1, 3)],
    [(5, 7), (2, 4), (7, 7)],
    [(-1, 0), (2, 1)],
    [(0, -1), (1, 2), [2, 0]],
    [(-10, 11), (2, 0), (1.5, 1), (11, -7), (-4, 10)]
]
labels = ValueArray([[1], [0], [0], [1], [0], [1]])
output = rnn(inputs[0])[-1]
output

Value(0.628636)

Next we'll define the training loop. We'll use cross entropy as loss function (as described in the last notebook). The RNN is many-to-one, so we'll only use the last output to make the prediction. The rest of the code is similar to the training loop we used last time.

In [15]:
# Training loop
for epoch in range(101):
    outputs = [[rnn(x)[-1]] for x in inputs]  # Many-to-one.
    loss = dlafs.loss.cross_entropy(labels, outputs)
    labels.zero_grad()
    loss.backward()
    for param in rnn.parameters():
        param.data -= 3e-1 * param.grad
    rnn.zero_grad()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, loss={loss.data:.3f}')

Epoch 0, loss=0.818
Epoch 10, loss=0.308
Epoch 20, loss=0.150
Epoch 30, loss=0.083
Epoch 40, loss=0.054
Epoch 50, loss=0.040
Epoch 60, loss=0.031
Epoch 70, loss=0.025
Epoch 80, loss=0.022
Epoch 90, loss=0.019
Epoch 100, loss=0.016


In [16]:
# Predictions
outputs = [rnn(x)[-1] for x in inputs]
for output, label in zip(outputs, labels):
    print(f'Prediction: {output.data:.3f}, label: {label.values[0].data}')

Prediction: 0.986, label: 1
Prediction: 0.015, label: 0
Prediction: 0.009, label: 0
Prediction: 0.974, label: 1
Prediction: 0.020, label: 0
Prediction: 0.989, label: 1


The RNN is able to fit the training data quite good, but since the dataset is so small, it will not be able to generalize to new samples. At least we know that it's working as intended.

In [17]:
x_test = [
    [[2, 5], [-30, -1]],
    [[1, 0], [0, 1], [1, 0], [0, 1]],
    [[0, 0], [1, 2], [7, 30]],
    ]
[rnn(x)[-1] for x in x_test]

[Value(0.997981), Value(0.074818), Value(0.964868)]

Before we end this notebook I'll show how we can create a one-to-many RNN. There are several ways to achieve this using our current architecture:

1. We can simply pass zeros as the input for all timesteps after the first one.
2. We can repeatedly pass the same input $x^{\braket{0}}$ for all timesteps.
3. We can pass the output $\hat{y}^{\braket{t}}$ (not $a^{\braket{t}}$) as the input for the next timestep.

I'll showcase this last approach since it's the most common. We'll create a new dataset where the input is a simple integer and the outputs is the next 5 integers. For instance, the input `10` should produce the output `[11, 12, 13, 14, 15]`s.

In [18]:
x = np.arange(0, 100, 1)
x = x.reshape(-1, 1, 1)  # (num_samples, seq_length, input_size) = (100, 1, 1)
x = ValueArray.from_numpy(x)
x[:5]

ValueArray(
    [[[0]],

     [[1]],

     [[2]],

     [[3]],

     [[4]]]
)

In [19]:
y = [np.arange(i+1, i+6).tolist() for i in range(0, 100)]
y = ValueArray(y)
y[:5]

ValueArray(
    [[1, 2, 3, 4, 5],
     [2, 3, 4, 5, 6],
     [3, 4, 5, 6, 7],
     [4, 5, 6, 7, 8],
     [5, 6, 7, 8, 9]]
)

We'll define a simple RNN which takes 1 input and has a hidden size of 4. The output layer will be a linear layer with 1 output.

In [20]:
layers = [
    RecurrentLayer(num_inputs=1, hidden_size=4, activation='linear'),
    Layer(4, 1, activation='linear')
]
rnn = RecurrentNN(layers)

Next, we'll create a function to generate predictions, where each prediction is fed as the input to the next timestep until we've reached `n_steps`.

In [21]:
def one_to_many_predict(model, x_0, n_steps):
    outputs = []
    x_i = x_0
    for _ in range(n_steps):
        y_hat_i = model(x_i)
        outputs.append(y_hat_i[-1])
        x_i = ValueArray([y_hat_i.values])
    return ValueArray(outputs)

one_to_many_predict(rnn, x[0], 5)[:5]

ValueArray(
    [-0.092553, -0.079858, -0.081599, -0.081361, -0.081393]
)

Lastly, we can define the training loop, which looks similar to the one we used previously, except that we're using the `one_to_many_predict` function to generate the predictions, and that we average the loss over all timesteps using MSE as a loss function.

In [22]:
# NB: takes a while to run
for epoch in range(101):
    outputs = ValueArray([one_to_many_predict(rnn, x_i, 5) for x_i in x])
    # Average loss over the 5 outputs per sample
    loss = sum(dlafs.loss.mse(y[:, i], outputs[:, i]) for i in range(5)) / 5
    labels.zero_grad()
    loss.backward()
    for param in rnn.parameters():
        param.data -= 7e-6 * param.grad
    rnn.zero_grad()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, loss={loss.data:.3f}')

Epoch 0, loss=3773.591
Epoch 10, loss=3653.840
Epoch 20, loss=3514.899
Epoch 30, loss=3322.788
Epoch 40, loss=2957.363
Epoch 50, loss=1276.207
Epoch 60, loss=2.771
Epoch 70, loss=2.765
Epoch 80, loss=2.760
Epoch 90, loss=2.754
Epoch 100, loss=2.748


The same function can be used to generate predictions of any length.

In [23]:
test_x = np.arange(100, 105).reshape(-1, 1, 1)
test_x = ValueArray.from_numpy(test_x)
preds = ValueArray([one_to_many_predict(rnn, x_i, 8) for x_i in test_x])
preds

ValueArray(
    [[101.470042, 102.961581, 104.474929, 106.010408, 107.568339, 109.149051, 110.752878, 112.380158],
     [102.484665,  103.99104, 105.519442, 107.070194, 108.643623, 110.240059, 111.859839, 113.503306],
     [103.499288, 105.020499, 106.563955, 108.129981, 109.718906, 111.331066, 112.966801, 114.626454],
     [104.513911, 106.049959, 107.608468, 109.189768,  110.79419, 112.422074, 114.073762, 115.749602],
     [105.528533, 107.079418, 108.652981, 110.249555, 111.869474, 113.513081, 115.180723,  116.87275]]
)

## Conclusion

We've created a recurrent neural network from scratch using only pure Python. Next up, we'll look at some of the limitations of RNNs, namely vanishing and exploding gradients, and how to address them using LSTMs.
